Adding columns

In [27]:
import pandas as pd

In [28]:
file_path = 'housing.csv'

data = pd.read_csv(file_path)

In [29]:
# Add a new column for price per square meter, avoiding division by zero or null landsize values
data['Land_price_per_m2'] = data.apply(
    lambda row: row['Price'] / row['Landsize'] if pd.notnull(row['Price']) and pd.notnull(row['Landsize']) and row['Landsize'] > 0 else None,
    axis=1
)

# Display the updated dataset's first few rows to confirm the addition
data[['Price', 'Landsize', 'Land_price_per_m2']].head()

,Price,Landsize,Land_price_per_m2
0,1480000.0,202.0,7326.732673
1,1035000.0,156.0,6634.615385
2,1465000.0,134.0,10932.835821
3,850000.0,94.0,9042.553191
4,1600000.0,120.0,13333.333333


In [30]:
# Add a new column for price per square meter, avoiding division by zero or null landsize values
data['Building_price_per_m2'] = data.apply(
    lambda row: row['Price'] / row['BuildingArea'] if pd.notnull(row['Price']) and pd.notnull(row['BuildingArea']) and row['BuildingArea'] > 0 else None,
    axis=1
)

# Display the updated dataset's first few rows to confirm the addition
data[['Price', 'BuildingArea', 'Building_price_per_m2']].head()

,Price,BuildingArea,Building_price_per_m2
0,1480000.0,NaN,NaN
1,1035000.0,79.0,13101.265823
2,1465000.0,150.0,9766.666667
3,850000.0,NaN,NaN
4,1600000.0,142.0,11267.605634


In [31]:
# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

data['Date'].head()

0   2016-12-03
1   2016-02-04
2   2017-03-04
3   2017-03-04
4   2016-06-04
Name: Date, dtype: datetime64[ns]

to check if the yearbuilt floating numbers, aside from the nan records, are all actual integers (so they're floating with .0 decimal):

In [32]:
all_integers = data['YearBuilt'].dropna().apply(lambda x: x.is_integer()).all()


print(f"All 'YearBuilt' values are integers with .0 floating point: {all_integers}")


All 'YearBuilt' values are integers with .0 floating point: True


In [33]:
data['YearBuilt'] = data['YearBuilt'].astype('Int64')

data['YearBuilt'].head()

0    <NA>
1    1900
2    1900
3    <NA>
4    2014
Name: YearBuilt, dtype: Int64

to check that also for property count, we are not converting values that might have a decimal part:

In [34]:
all_integers_property_count = data['Propertycount'].apply(lambda x: x.is_integer()).all()

print(f"All 'Propertycount' values are integers with .0 floating point: {all_integers_property_count}")

All 'Propertycount' values are integers with .0 floating point: True


In [36]:
data['Propertycount'] = data['Propertycount'].astype('Int64')

data['Propertycount'].head()


0    4019
1    4019
2    4019
3    4019
4    4019
Name: Propertycount, dtype: Int64

(i still have to understand wtf is propertycount...)

next: the dummy vars for the regions :)